In [6]:
import yfinance as yf
import pandas as pd
import numpy as np

In [5]:
def get_historical_data(tickers, start_date, end_date):
    """
    Retrieve historical stock prices for a list of tickers.
    
    Args:
    - tickers (list): List of stock tickers (e.g., ["AAPL", "MSFT"]).
    - start_date (str): Start date in the format "YYYY-MM-DD".
    - end_date (str): End date in the format "YYYY-MM-DD".
    
    Returns:
    - DataFrame: DataFrame containing historical stock prices for the specified tickers and date range.
    """
    # Create an empty list to store the historical dataframes
    historical_data = []
    
    # Iterate over each ticker and fetch historical data
    for ticker in tickers:
        # Fetch historical data from Yahoo Finance
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        
        # Add a column to identify the stock ticker
        stock_data["Ticker"] = ticker
        
        # Append the data to the historical_data list
        historical_data.append(stock_data)
    
    # Concatenate the dataframes in the list into a single dataframe
    combined_data = pd.concat(historical_data)
    
    return combined_data

# Example usage
tickers = ["AAPL", "MSFT", "GOOGL"]
start_date = "2020-01-01"
end_date = "2021-01-01"
historical_prices = get_historical_data(tickers, start_date, end_date)

# Display the first few rows of the DataFrame
print(historical_prices.head())



c:\Users\willb\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
c:\Users\willb\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
c:\Users\willb\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd

                 Open       High        Low      Close  Adj Close     Volume  \
Date                                                                           
2020-01-02  74.059998  75.150002  73.797501  75.087502  73.059433  135480400   
2020-01-03  74.287498  75.144997  74.125000  74.357498  72.349144  146322800   
2020-01-06  73.447502  74.989998  73.187500  74.949997  72.925644  118387200   
2020-01-07  74.959999  75.224998  74.370003  74.597504  72.582672  108872000   
2020-01-08  74.290001  76.110001  74.290001  75.797501  73.750237  132079200   

           Ticker  
Date               
2020-01-02   AAPL  
2020-01-03   AAPL  
2020-01-06   AAPL  
2020-01-07   AAPL  
2020-01-08   AAPL  


In [16]:
class Portfolio:
    def __init__(self, tickers, start_date, end_date, initial_weights=None):
        """
        Initialize a portfolio with given assets and weights.
        
        Args:
        - tickers (list): List of stock tickers (e.g., ["AAPL", "MSFT"]).
        - start_date (str): Start date in the format "YYYY-MM-DD".
        - end_date (str): End date in the format "YYYY-MM-DD".
        - initial_weights (dict, optional): Initial weights for each asset (default: equal weights).
        """
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        
        # Initialize portfolio weights
        if initial_weights is None:
            self.weights = {ticker: 1/len(tickers) for ticker in tickers}
        else:
            self.weights = initial_weights
        
        # Retrieve historical data
        self.historical_data = self._retrieve_historical_data()
        
        # Calculate daily returns
        self.daily_returns = self._calculate_daily_returns()
        
        # Calculate portfolio value
        self.portfolio_value = self._calculate_portfolio_value()
        
        # Calculate portfolio statistics
        self.statistics = self._calculate_statistics()
    
    def _retrieve_historical_data(self):
        """Retrieve historical stock prices for the portfolio assets."""
        # Fetch historical data for the tickers
        data = yf.download(self.tickers, start=self.start_date, end=self.end_date)
        
        # Drop any NaN values
        data.dropna(inplace=True)
        
        # Return the DataFrame
        return data
    
    def _calculate_daily_returns(self):
        """Calculate daily returns for each asset."""
        # Calculate daily percentage change in prices
        daily_returns = self.historical_data['Adj Close'].pct_change()

        # Drop any NaN values
        self.historical_data.dropna(inplace=True)
        
        return daily_returns
    
    def _calculate_portfolio_value(self):
        """Calculate the portfolio value over time based on historical data and weights."""
        # Calculate daily portfolio values
        daily_portfolio_values = (self.historical_data['Adj Close'] * pd.Series(self.weights)).sum(axis=1)
        
        return daily_portfolio_values
    
    def _calculate_statistics(self):
        """Calculate portfolio statistics such as total return, volatility, and Sharpe ratio."""
        # Calculate total return
        total_return = (self.portfolio_value.iloc[-1] / self.portfolio_value.iloc[0]) - 1
        
        # Calculate annualized volatility (standard deviation of daily returns)
        annualized_volatility = self.daily_returns.std() * np.sqrt(252)
        
        # Calculate annualized Sharpe ratio
        risk_free_rate = 0.02  # Example risk-free rate
        excess_returns = self.daily_returns - risk_free_rate / 252
        annualized_sharpe_ratio = excess_returns.mean() / excess_returns.std() * np.sqrt(252)
        
        statistics = {
            "Total Return": total_return,
            "Annualized Volatility": annualized_volatility,
            "Annualized Sharpe Ratio": annualized_sharpe_ratio
        }
        
        return statistics

tickers = ["AAPL", "MSFT", "GOOGL"]
start_date = "2020-01-01"
end_date = "2021-01-01"
portfolio = Portfolio(tickers, start_date, end_date)

# Access portfolio attributes
print("Tickers:", portfolio.tickers)
print("Weights:", portfolio.weights)
print("Historical Data:")
print(portfolio.historical_data.head())
print("Daily Returns:")
print(portfolio.daily_returns.head())
print("Portfolio Value:")
print(portfolio.portfolio_value.head())
print("Statistics:")
print(portfolio.statistics)


c:\Users\willb\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
c:\Users\willb\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[                       0%%                      ]

c:\Users\willb\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  3 of 3 completed

Tickers: ['AAPL', 'MSFT', 'GOOGL']
Weights: {'AAPL': 0.3333333333333333, 'MSFT': 0.3333333333333333, 'GOOGL': 0.3333333333333333}
Historical Data:
Price       Adj Close                             Close             \
Ticker           AAPL      GOOGL        MSFT       AAPL      GOOGL   
Date                                                                 
2020-01-02  73.059433  68.433998  154.493851  75.087502  68.433998   
2020-01-03  72.349144  68.075996  152.570114  74.357498  68.075996   
2020-01-06  72.925644  69.890503  152.964462  74.949997  69.890503   
2020-01-07  72.582672  69.755501  151.569778  74.597504  69.755501   
2020-01-08  73.750237  70.251999  153.984039  75.797501  70.251999   

Price                        High                               Low  \
Ticker            MSFT       AAPL      GOOGL        MSFT       AAPL   
Date                                                                  
2020-01-02  160.619995  75.150002  68.433998  160.729996  73.797501   
2020-01-

In [9]:
portfolio.tickers

NameError: name 'portfolio' is not defined